In [1]:
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np

In [2]:
import pandas as pd
from PIL import Image
import xml.etree.ElementTree as ET

def create_DF(files, images_dir):
    """
    Convierte una lista de PosixPaths en un dataframe
    """
    df = pd.DataFrame([])
    for file in files:
        tree = ET.parse(file)
        root = tree.getroot()

        image_dic = []

        for obj in root.findall('object'):
            label = obj.find('name').text       # pineapple
            image = file.stem + ".png"          # fruit0.png
            bndbox = obj.find('bndbox')         
            xmin = float(bndbox.find('xmin').text)      # 38
            ymin = float(bndbox.find('ymin').text)     # 18
            xmax = float(bndbox.find('xmax').text)     # 271)
            ymax = float(bndbox.find('ymax').text)      # 227)

            image_dic.append({
                "label" : label,
                "left" : xmin, 
                "top" : ymin, 
                "right" : xmax,
                "bottom" : ymax
            })
        
        aux_df = pd.DataFrame(image_dic)
        
        # try:
        image_file = next(images_dir.glob(f"**/{file.stem}*.png"))
        aux_df["height"] = float(root.find('size').find('height').text)
        aux_df["width"] = float(root.find('size').find('width').text)
        aux_df["image"] = image_file
        aux_df["image_name"] = root.find('filename').text
        df = pd.concat([df, aux_df])
        # except:
        #     print(file.stem)
    print(df)
    df["center_x"] = (df["right"] + df["left"])/2
    df["center_y"] = (df["bottom"] + df["top"])/2
    df["delta_x"] = df["right"] - df["left"]
    df["delta_y"] = df["bottom"] - df["top"]

    return df

In [3]:
train_files = list(Path("dataset/annotations/").glob("**/*xml"))
train_files2 = list(Path("dataset/annotations/").glob("**/*xml"))
# val_files = list(Path("dataset/annotations/").glob("**/*xml"))

print(train_files)

[WindowsPath('dataset/annotations/fruit0.xml'), WindowsPath('dataset/annotations/fruit1.xml'), WindowsPath('dataset/annotations/fruit10.xml'), WindowsPath('dataset/annotations/fruit100.xml'), WindowsPath('dataset/annotations/fruit101.xml'), WindowsPath('dataset/annotations/fruit102.xml'), WindowsPath('dataset/annotations/fruit103.xml'), WindowsPath('dataset/annotations/fruit104.xml'), WindowsPath('dataset/annotations/fruit105.xml'), WindowsPath('dataset/annotations/fruit106.xml'), WindowsPath('dataset/annotations/fruit107.xml'), WindowsPath('dataset/annotations/fruit108.xml'), WindowsPath('dataset/annotations/fruit109.xml'), WindowsPath('dataset/annotations/fruit11.xml'), WindowsPath('dataset/annotations/fruit110.xml'), WindowsPath('dataset/annotations/fruit111.xml'), WindowsPath('dataset/annotations/fruit112.xml'), WindowsPath('dataset/annotations/fruit113.xml'), WindowsPath('dataset/annotations/fruit114.xml'), WindowsPath('dataset/annotations/fruit115.xml'), WindowsPath('dataset/anno

In [4]:
df_train = create_DF(train_files, Path("dataset/images/"))

           label   left    top  right  bottom  height  width  \
0      pineapple   38.0   82.0  271.0   227.0   300.0  400.0   
1    snake fruit  244.0  174.0  280.0   207.0   300.0  400.0   
2   dragon fruit  254.0  228.0  351.0   300.0   300.0  400.0   
0      pineapple   38.0   87.0  275.0   241.0   300.0  400.0   
1    snake fruit  240.0  185.0  279.0   220.0   300.0  400.0   
..           ...    ...    ...    ...     ...     ...    ...   
1    snake fruit  144.0  181.0  162.0   204.0   300.0  400.0   
2         banana  204.0  217.0  253.0   262.0   300.0  400.0   
0      pineapple  155.0   80.0  261.0   175.0   300.0  400.0   
1    snake fruit  143.0  184.0  161.0   208.0   300.0  400.0   
2         banana  203.0  222.0  257.0   269.0   300.0  400.0   

                         image   image_name  
0    dataset\images\fruit0.png   fruit0.png  
1    dataset\images\fruit0.png   fruit0.png  
2    dataset\images\fruit0.png   fruit0.png  
0    dataset\images\fruit1.png   fruit1.png  
1

In [5]:
# df_val = create_DF(val_files,Path("Dataset/validation/Apple_Grape_Cantaloupe_Watermelon_Pomegranate_Pineapple_Peach_Mango_Banana_Lemon_Tomato_Strawberry_Pear/"))

In [6]:
df_train.head()

,label,left,top,right,bottom,height,width,image,image_name,center_x,center_y,delta_x,delta_y
0,pineapple,38.0,82.0,271.0,227.0,300.0,400.0,dataset\images\fruit0.png,fruit0.png,154.5,154.5,233.0,145.0
1,snake fruit,244.0,174.0,280.0,207.0,300.0,400.0,dataset\images\fruit0.png,fruit0.png,262.0,190.5,36.0,33.0
2,dragon fruit,254.0,228.0,351.0,300.0,300.0,400.0,dataset\images\fruit0.png,fruit0.png,302.5,264.0,97.0,72.0
0,pineapple,38.0,87.0,275.0,241.0,300.0,400.0,dataset\images\fruit1.png,fruit1.png,156.5,164.0,237.0,154.0
1,snake fruit,240.0,185.0,279.0,220.0,300.0,400.0,dataset\images\fruit1.png,fruit1.png,259.5,202.5,39.0,35.0


In [7]:
valores_unicos_columna2 = df_train['image_name'].unique()

np.random.shuffle(valores_unicos_columna2)

df_val = df_train[df_train['image_name'].isin(valores_unicos_columna2[:20])]
df_train = df_train[~df_train['image_name'].isin(valores_unicos_columna2[:20])]

In [8]:
df_val.head(100)

,label,left,top,right,bottom,height,width,image,image_name,center_x,center_y,delta_x,delta_y
0,pineapple,46.0,85.0,222.0,185.0,300.0,400.0,dataset\images\fruit104.png,fruit104.png,134.0,135.0,176.0,100.0
1,snake fruit,202.0,202.0,238.0,225.0,300.0,400.0,dataset\images\fruit104.png,fruit104.png,220.0,213.5,36.0,23.0
2,banana,277.0,117.0,332.0,178.0,300.0,400.0,dataset\images\fruit104.png,fruit104.png,304.5,147.5,55.0,61.0
0,pineapple,115.0,90.0,261.0,249.0,300.0,400.0,dataset\images\fruit115.png,fruit115.png,188.0,169.5,146.0,159.0
1,snake fruit,288.0,148.0,311.0,168.0,300.0,400.0,dataset\images\fruit115.png,fruit115.png,299.5,158.0,23.0,20.0
2,banana,219.0,65.0,271.0,111.0,300.0,400.0,dataset\images\fruit115.png,fruit115.png,245.0,88.0,52.0,46.0
0,pineapple,128.0,122.0,235.0,269.0,300.0,400.0,dataset\images\fruit121.png,fruit121.png,181.5,195.5,107.0,147.0
1,snake fruit,260.0,168.0,278.0,188.0,300.0,400.0,dataset\images\fruit121.png,fruit121.png,269.0,178.0,18.0,20.0
2,banana,205.0,79.0,251.0,123.0,300.0,400.0,dataset\images\fruit121.png,fruit121.png,228.0,101.0,46.0,44.0
0,pineapple,168.0,127.0,345.0,240.0,300.0,400.0,dataset\images\fruit130.png,fruit130.png,256.5,183.5,177.0,113.0


In [9]:
classes = df_train["label"].unique()
classes

array(['pineapple', 'snake fruit', 'dragon fruit', 'banana'], dtype=object)

In [10]:
classes_idx = {clase:idx for idx,clase in enumerate(classes)}

In [11]:
cant_classes = len(classes)
cant_classes

4

In [12]:
train_files = list(Path("dataset/annotations/").glob("**/*xml"))
# val_files = list(Path("dataset/annotations/").glob("**/*xml"))

print(train_files)

[WindowsPath('dataset/annotations/fruit0.xml'), WindowsPath('dataset/annotations/fruit1.xml'), WindowsPath('dataset/annotations/fruit10.xml'), WindowsPath('dataset/annotations/fruit100.xml'), WindowsPath('dataset/annotations/fruit101.xml'), WindowsPath('dataset/annotations/fruit102.xml'), WindowsPath('dataset/annotations/fruit103.xml'), WindowsPath('dataset/annotations/fruit104.xml'), WindowsPath('dataset/annotations/fruit105.xml'), WindowsPath('dataset/annotations/fruit106.xml'), WindowsPath('dataset/annotations/fruit107.xml'), WindowsPath('dataset/annotations/fruit108.xml'), WindowsPath('dataset/annotations/fruit109.xml'), WindowsPath('dataset/annotations/fruit11.xml'), WindowsPath('dataset/annotations/fruit110.xml'), WindowsPath('dataset/annotations/fruit111.xml'), WindowsPath('dataset/annotations/fruit112.xml'), WindowsPath('dataset/annotations/fruit113.xml'), WindowsPath('dataset/annotations/fruit114.xml'), WindowsPath('dataset/annotations/fruit115.xml'), WindowsPath('dataset/anno

In [13]:
# df_val = create_DF(val_files,Path("Dataset/validation/Apple_Grape_Cantaloupe_Watermelon_Pomegranate_Pineapple_Peach_Mango_Banana_Lemon_Tomato_Strawberry_Pear/"))

In [14]:
df_val.head()

,label,left,top,right,bottom,height,width,image,image_name,center_x,center_y,delta_x,delta_y
0,pineapple,46.0,85.0,222.0,185.0,300.0,400.0,dataset\images\fruit104.png,fruit104.png,134.0,135.0,176.0,100.0
1,snake fruit,202.0,202.0,238.0,225.0,300.0,400.0,dataset\images\fruit104.png,fruit104.png,220.0,213.5,36.0,23.0
2,banana,277.0,117.0,332.0,178.0,300.0,400.0,dataset\images\fruit104.png,fruit104.png,304.5,147.5,55.0,61.0
0,pineapple,115.0,90.0,261.0,249.0,300.0,400.0,dataset\images\fruit115.png,fruit115.png,188.0,169.5,146.0,159.0
1,snake fruit,288.0,148.0,311.0,168.0,300.0,400.0,dataset\images\fruit115.png,fruit115.png,299.5,158.0,23.0,20.0


In [15]:
classes_idx = {clase:idx for idx,clase in enumerate(classes)}

In [16]:
import pickle
with open("data.pkl","wb") as f:
    pickle.dump((df_train, df_val, train_files, classes_idx, cant_classes, classes),f)